#### 步骤:
1. 读取测试训练的t3,t4,t5,t6.(如需要天气数据则包括t7)
2. 按照20分钟时间窗口得到 t5_train,t6_train,t5_test_2h_ago,t6_testt_2h_ago
3. t5_test_2h_ago,t6_testt_2h_ago,时间窗口加2小时去掉标签,得到t5_test,t6_test
4. 按照时间日期给t5_train,t6_train,t5_test,t6_test,加特征是否假期,星期几,一天中第几个时间窗口
5. 清洗t4,t5得每条路线的,宽度长度岔路数,t5_train,t5_test按照Intersection 和 Tollgates 添加路途宽度长度岔路数特征
6. t5_train,t6_train,t5_test,t6_test数据格式修改到要求的格式,保存到csv文件中
7. 查看文件发现并修改一些问题

## step1 读取数据

In [29]:
import pandas as pd 
import glob
t3 = pd.read_csv('dataSets/training/links (table 3).csv')
t4 = pd.read_csv('dataSets/training/routes (table 4).csv')
t5_train = pd.read_csv('dataSets/training/trajectories(table 5)_training.csv')
t6_train = pd.read_csv(glob.glob("dataSets/training/*.csv")[-1])


In [30]:
t6_test_2h_ago = pd.read_csv(glob.glob("dataSets/testing_phase1/*.csv")[0])
t5_test_2h_ago = pd.read_csv(glob.glob("dataSets/testing_phase1/*.csv")[-1])


In [31]:
t5_test_2h_ago.head()

,intersection_id,tollgate_id,vehicle_id,starting_time,travel_seq,travel_time
0,A,2,1026631,2016-10-18 06:00:14,110#2016-10-18 06:00:14#7.65;123#2016-10-18 06...,27.54
1,C,3,1017312,2016-10-18 06:00:28,115#2016-10-18 06:00:28#9.35;102#2016-10-18 06...,139.53
2,A,2,1044671,2016-10-18 06:03:07,110#2016-10-18 06:03:07#7.22;123#2016-10-18 06...,26.01
3,A,3,1000017,2016-10-18 06:04:37,110#2016-10-18 06:04:37#7.39;123#2016-10-18 06...,78.40
4,A,2,1002850,2016-10-18 06:10:17,110#2016-10-18 06:10:17#14.33;123#2016-10-18 0...,71.00


## step2 按照20分钟窗口重组数据

In [32]:
# 按照时间窗口重组数据: http://stackoverflow.com/questions/36914892/python-how-to-group-pandas-data-frame-in-a-certain-time-window
t5_train['starting_time'] =  pd.to_datetime(t5_train['starting_time'] , format='%Y-%m-%d %H:%M:%S')
t5_train = t5_train.set_index(['starting_time'])

t5_train = t5_train.groupby([pd.TimeGrouper('20Min'), 'intersection_id', 'tollgate_id']).travel_time.mean()\
       .reset_index().rename(columns = {'travel_time':'avg_travel_time'})

In [33]:
t5_test_2h_ago['starting_time'] =  pd.to_datetime(t5_test_2h_ago['starting_time'] , format='%Y-%m-%d %H:%M:%S')
t5_test_2h_ago = t5_test_2h_ago.set_index(['starting_time'])

t5_test_2h_ago = t5_test_2h_ago.groupby([pd.TimeGrouper('20Min'), 'intersection_id', 'tollgate_id']).travel_time.mean()\
       .reset_index().rename(columns = {'travel_time':'avg_travel_time'})

In [34]:
t6_train.head(5)

,time,tollgate_id,direction,vehicle_model,has_etc,vehicle_type
0,2016-09-19 23:09:25,2,0,1,0,NaN
1,2016-09-19 23:11:53,2,0,1,0,NaN
2,2016-09-19 23:13:54,2,0,1,0,NaN
3,2016-09-19 23:17:48,1,0,1,1,NaN
4,2016-09-19 23:16:07,2,0,1,0,NaN


In [35]:
t6_train['time'] =  pd.to_datetime(t6_train['time'] , format='%Y-%m-%d %H:%M:%S')
t6_train = t6_train.set_index(['time'])

# 车流量
t6_train = t6_train.groupby([pd.TimeGrouper('20Min'), 'tollgate_id', 'direction']).size()\
       .reset_index().rename(columns = {0:'volume'})

,time,tollgate_id,direction,volume
0,2016-09-19,1,0,13
1,2016-09-19,1,1,140
2,2016-09-19,2,0,2
3,2016-09-19,3,0,17
4,2016-09-19,3,1,181


In [36]:
t6_test_2h_ago['time'] =  pd.to_datetime(t6_test_2h_ago['time'] , format='%Y-%m-%d %H:%M:%S')
t6_test_2h_ago = t6_test_2h_ago.set_index(['time'])

# 车流量
t6_test_2h_ago = t6_test_2h_ago.groupby([pd.TimeGrouper('20Min'), 'tollgate_id', 'direction']).size()\
       .reset_index().rename(columns = {0:'volume'})

## step3 生成测试数据

In [42]:
# apped 2h前测试数据
t5_train= t5_train.append(t5_test_2h_ago)
t6_train= t6_train.append(t6_test_2h_ago)

In [45]:
# 增加两小时作为测试数据
t5_test_2h_ago['starting_time'] = t5_test_2h_ago['starting_time'] + pd.DateOffset(hours=2)
t6_test_2h_ago['time'] = t6_test_2h_ago['time'] + pd.DateOffset(hours=2)

In [48]:
# 测试数据
t5_test,t6_test =t5_test_2h_ago,t6_test_2h_ago
del t5_test_2h_ago,t6_test_2h_ago

In [51]:
# 移除标签
t5_test.drop('avg_travel_time',axis=1,inplace=True)
t6_test.drop('volume',axis=1,inplace=True)

## step4 生成日期特征

In [73]:
t5_test['starting_time'].dt.date.loc[0]

datetime.date(2016, 10, 18)

In [74]:
# 中秋国庆假期

from datetime import datetime
start1 = datetime(2016, 9, 15)
end1 = datetime(2016, 9, 17)
start2 = datetime(2016, 10, 1)
end2 = datetime(2016, 10, 7)
rng = pd.date_range(start1, end1).append(pd.date_range(start2, end2))

# 是否假期,星期,小时,在那个时间窗口

def generateDataTime(X,label):
    
    # 小时,星期,时间窗口
    hour = pd.get_dummies(X[label].dt.hour,prefix='hour_')
    weekday = pd.get_dummies(X[label].dt.weekday_name)
    time = pd.get_dummies(X[label].dt.time)
    # 添加到X
    X = pd.concat([X,weekday,hour,time], axis=1)
    # 是否在假期内
    date = X[label].dt.date
    for i, row in X.iterrows():
        X.loc[i, "holoday"] = 0
        if date.loc[i] in rng: X.loc[i, "holoday"] = 1
    return X

In [77]:
# 生成包含日期数据的数据
t5_test = generateDataTime(t5_test,'starting_time')
t6_test = generateDataTime(t6_test,'time')
t5_train = generateDataTime(t5_train,'starting_time')
t6_train = generateDataTime(t6_train,'time')

## step5 每条路径特征提取

In [78]:
# 将110,123 变成[110,123] 
def split(str): return str.split(",")
t4.link_seq = t4.link_seq.apply(split)

# 将数据按link_seq中各link展开
rows = []
_ = t4.apply(lambda row: [rows.append([row['intersection_id'], row['tollgate_id'], nn]) 
                         for nn in row.link_seq], axis=1)
col = ['intersection_id', 'tollgate_id','link_id']
t4_new = pd.DataFrame(rows, columns=col)

In [80]:
t3.head()

,link_id,length,width,lanes,in_top,out_top,lane_width
0,100,58,3,1,105,111,3
1,101,84,3,1,116,121,3
2,102,131,9,3,115,109,3
3,103,23,12,4,111,"122,116",3
4,104,293,9,3,109,112,3


In [81]:
# pd.merge(t4_new,t3,on = 'link_id',how='left')
# 提取每条link的是否有in,out交叉路口
t3['cross_in'] = 0
t3['cross_out'] = 0

for i, row in t3.iterrows():
    if ',' in str(row['in_top']):
        t3.loc[i, "cross_in"] = 1
    if ',' in str(row['out_top']):
        t3.loc[i, "cross_out"] = 1

In [82]:
t3['link_id'] = t3['link_id'].astype(str)
t4_new['link_id'] = t4_new['link_id'].astype(str)
t4_new = pd.merge(t4_new,t3,on = 'link_id',how='left')
t4_new.drop(['in_top','out_top'],axis =1,inplace= True)

In [84]:
# 聚合t4_new
# in out 岔路口数
b1 = t4_new[['intersection_id', 'tollgate_id','cross_in']]\
           .groupby(['intersection_id', 'tollgate_id'])\
            .cross_in.sum().reset_index().rename(columns = {'cross_in':'in_link_cross_count'})
b2 = t4_new[['intersection_id', 'tollgate_id','cross_out']]\
        .groupby(['intersection_id', 'tollgate_id'])\
        .cross_out.sum().reset_index().rename(columns = {'cross_out':'out_link_cross_count'})
road = pd.merge(b1,b2,on =['intersection_id', 'tollgate_id'] ,how='left')

# 路程
b1 = t4_new[['intersection_id', 'tollgate_id','length']].groupby(['intersection_id', 'tollgate_id']).length.sum().reset_index()
road = pd.merge(road,b1,on =['intersection_id', 'tollgate_id'] ,how='left')
## 车道数1车道2车道3车道4车道的link数，后期考虑：占总路程的比率
# 各个路径的link总数
b1 = t4_new[['intersection_id', 'tollgate_id']]\
       .groupby(['intersection_id', 'tollgate_id']).size()\
    .reset_index().rename(columns = {0:'link_count'})
road = pd.merge(road,b1,on =['intersection_id', 'tollgate_id'] ,how='left')

In [86]:
# 1,2,3,4车道道路长度
b1 = t4_new[t4_new.lanes== 1][['intersection_id', 'tollgate_id','length']]\
       .groupby(['intersection_id', 'tollgate_id']).length.sum()\
    .reset_index().rename(columns = {'length':'1_length'})
road = pd.merge(road,b1,on =['intersection_id', 'tollgate_id'] ,how='left')
# 2车道路长度
b1 = t4_new[t4_new.lanes== 2][['intersection_id', 'tollgate_id','length']]\
       .groupby(['intersection_id', 'tollgate_id']).length.sum()\
    .reset_index().rename(columns = {'length':'2_length'})
road = pd.merge(road,b1,on =['intersection_id', 'tollgate_id'] ,how='left')
# 3车道路长度
b1 = t4_new[t4_new.lanes== 3][['intersection_id', 'tollgate_id','length']]\
       .groupby(['intersection_id', 'tollgate_id']).length.sum()\
    .reset_index().rename(columns = {'length':'3_length'})
road = pd.merge(road,b1,on =['intersection_id', 'tollgate_id'] ,how='left')
# 4车道路长度
b1 = t4_new[t4_new.lanes== 4][['intersection_id', 'tollgate_id','length']]\
       .groupby(['intersection_id', 'tollgate_id']).length.sum()\
    .reset_index().rename(columns = {'length':'4_length'})
road = pd.merge(road,b1,on =['intersection_id', 'tollgate_id'] ,how='left')


In [87]:
## 1,2,3,4车道link数
b1 = t4_new[t4_new.lanes== 1][['intersection_id', 'tollgate_id']]\
       .groupby(['intersection_id', 'tollgate_id']).size()\
    .reset_index().rename(columns = {0:'1_count'})
road = pd.merge(road,b1,on =['intersection_id', 'tollgate_id'] ,how='left')
# 2车道link数
b1 = t4_new[t4_new.lanes== 2][['intersection_id', 'tollgate_id']]\
       .groupby(['intersection_id', 'tollgate_id']).size()\
    .reset_index().rename(columns = {0:'2_count'})
road = pd.merge(road,b1,on =['intersection_id', 'tollgate_id'] ,how='left')
# 3车道link数
b1 = t4_new[t4_new.lanes== 3][['intersection_id', 'tollgate_id']]\
       .groupby(['intersection_id', 'tollgate_id']).size()\
    .reset_index().rename(columns = {0:'3_count'})
road = pd.merge(road,b1,on =['intersection_id', 'tollgate_id'] ,how='left')
# 4车道link数
b1 = t4_new[t4_new.lanes== 4][['intersection_id', 'tollgate_id']]\
       .groupby(['intersection_id', 'tollgate_id']).size()\
    .reset_index().rename(columns = {0:'4_count'})
road = pd.merge(road,b1,on =['intersection_id', 'tollgate_id'] ,how='left')

road.fillna(0,inplace=True)


In [89]:
t5_test = pd.merge(t5_test,road,on =['intersection_id', 'tollgate_id'] ,how='left')
t5_train = pd.merge(t5_train,road,on =['intersection_id', 'tollgate_id'] ,how='left')

## step 6 数据格式修改到要求的格式,保存到csv

In [93]:
t5_test['end'] = t5_test['starting_time'] + pd.DateOffset(minutes=20)
t5_train['end'] = t5_train['starting_time'] + pd.DateOffset(minutes=20)
t6_test['end'] = t6_test['time'] + pd.DateOffset(minutes=20)
t6_train['end'] = t6_train['time'] + pd.DateOffset(minutes=20)

In [101]:
# http://stackoverflow.com/questions/30132282/datetime-to-string-with-series-in-python-pandas
def timewindows(X,start,end):
    s= X[start].apply(lambda x: x.strftime('%Y-%m-%d %H:%M:%S'))
    e = X[end].apply(lambda x: x.strftime('%Y-%m-%d %H:%M:%S'))
    X['time_window'] = '['+ s +','+ e +')'
    return X.drop([start,end],axis =1)

In [103]:
t5_test = timewindows(t5_test,'starting_time','end')
t5_train = timewindows(t5_train,'starting_time','end')
t6_test = timewindows(t6_test,'time','end')
t6_train = timewindows(t6_train,'time','end')

In [105]:
t6_test.columns.values

array(['tollgate_id', 'direction', 'Friday', 'Monday', 'Saturday',
       'Sunday', 'Thursday', 'Tuesday', 'Wednesday', 'hour__8', 'hour__9',
       'hour__17', 'hour__18', datetime.time(8, 0), datetime.time(8, 20),
       datetime.time(8, 40), datetime.time(9, 0), datetime.time(9, 20),
       datetime.time(9, 40), datetime.time(17, 0), datetime.time(17, 20),
       datetime.time(17, 40), datetime.time(18, 0), datetime.time(18, 20),
       datetime.time(18, 40), 'holoday', 'time_window'], dtype=object)

In [106]:
t5_test = t5_test.set_index(['intersection_id','tollgate_id','time_window'])
t5_train = t5_train.set_index(['intersection_id','tollgate_id','time_window'])
t6_test = t6_test.set_index(['tollgate_id','time_window', 'direction'])
t6_train = t6_train.set_index(['tollgate_id','time_window', 'direction'])

In [110]:
t5_test.to_csv('dataSets/big_testset_task1.csv')
t5_train.to_csv('dataSets/big_trainset_task1.csv')
t6_test.to_csv('dataSets/big_testset_task2.csv')
t6_train.to_csv('dataSets/big_trainset_task2.csv')

## step7 修复问题
* 发现测试和训练数据时间维度不同
* 原因：pd.getdummy()是吧所有可能取值都作为一列，但是测试数据时间的取值范围要比训练集小，顾会少很多列
* 办法：补充这些列

In [124]:
t5_testcol,t5_traincol = list(t5_test.columns.values),list(t5_train.columns.values)
t6_testcol,t6_traincol = list(t6_test.columns.values),list(t6_train.columns.values)
mis_5 =  [x for x in t5_traincol  if x not in t5_testcol]
mis_6 =  [x for x in t6_traincol  if x not in t6_testcol]

for label in mis_5:
    t5_test[label] = 0
for label in mis_6:
    t6_test[label] = 0


In [127]:
# 调整各列顺序
t5_test = t5_test[t5_traincol]
t6_test = t6_test[t6_traincol]

In [132]:
t5_test.to_csv('dataSets/big_testset_task1.csv')
t5_train.to_csv('dataSets/big_trainset_task1.csv')
t6_test.to_csv('dataSets/big_testset_task2.csv')
t6_train.to_csv('dataSets/big_trainset_task2.csv')